<a href="https://colab.research.google.com/github/pvh95/Uni/blob/master/YSOUT0_AA2_Gauss_and_LU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Gauss-elimination**

In [ ]:
import numpy as np 
import sympy 
import scipy.sparse as sp
import scipy.sparse.linalg
from numpy.linalg import matrix_rank
import copy

In [ ]:
def gauss_elim(mtx, b): 
  ''' 
  Expected to have a square mtx as an input
  '''
  mtx = mtx.astype(np.float64)
  b = b.astype(np.float64)

  #print(mtx.shape)
  #print()

  #### If not a square mtx, throw an error message
  if mtx.shape[0] != mtx.shape[1]:
    raise TypeError('Please provide a square matrix')

  if mtx.shape[0]!= len(b):
    raise TypeError('The dimension of b is not inline with the number of rows of your matrix.')

  n = mtx.shape[0] #Extract dimension
  mtx_exp = np.zeros((n,n+1)) #Creating Augumented/Expanded matrix and filling them w/ a zero
  mtx_exp[:, :n] = mtx  #Original matrix to the nxn area
  mtx_exp[:, n] = b     #Append the n+1th column with b
  #print(mtx_exp)

  #### Gauss can be implemented iff r(mtx) = r(mtx|b)
  if matrix_rank(mtx) != matrix_rank(mtx_exp): 
    raise TypeError('Please provide another square mtx, because yours will not be solved due to the rank problem.')

  for i in range(n): #Using pivot selection algorithm w/ regards to column
    #pivot_val = max(mtx[i:, i]) #Determine the biggest value in the RIGHT size of column 
    #row_index = np.where(mtx[i:, i] == pivot_val)[0][0] + i   #As we decrease the size of a matrix when selecting from the reduced column, to get the real row_index of the biggest element => need to append it with the reduced dimension
    abs_array = np.absolute(mtx[i:, i])   #Taking of abs values of each elemnts
    pivot_val = max(abs_array)   # select the  biggest value
    row_index = np.where(abs_array == pivot_val)[0][0] + i   #As we decrease the size of a matrix when selecting from the reduced column, to get the real row_index of the biggest element => need to append it with the reduced dimension
    #print('Before tinkering:')
    #print(mtx)
    #print(b)
    #print()
    mtx[[i,row_index]] = mtx[[row_index, i]]   #swap rows according to row_index determined by the biggest element in a column 
    b[i], b[row_index] = b[row_index], b[i]    #swap corresponding to the appropriate elements 
    #print(mtx)
    #print(b)
    #print()

    mtx[i, :] = mtx[i, :]/pivot_val        #To get 1 at the pivot element, need to divide the whole row by the pivot_element
    b[i] = b[i]/pivot_val                   #Do the same with the corresponding b value 

    #print(mtx)
    #print(b)
    #print()

    ###The Gauss-elimination step 
    for j in range(i+1, n):                 
      ratio = mtx[j,i]/mtx[i,i]
      #print(ratio)                 
      mtx[j] = mtx[j] - ratio*mtx[i]
      b[j] = b[j] - ratio*b[i]

    #print(mtx)
    #print(b)
    #print('---------------------------')


  x = np.zeros(b.shape)         #creating the zero_vector x having the same shape as the b vector

  #Backsubtitution:
  x[n-1] = b[n-1]/mtx[n-1, n-1]     #Calculating the last value of x with the help of the corresponding b value and the bottom right element of the upper triangular matrix
  #print(x)


  for i in range(n-2, -1, -1):
    x[i] = b[i]

    for j in range(i+1, n):
      x[i] = x[i] - mtx[i, j]*x[j]     #Subtracting the appropriate values from the matrix with the known coefficents x calculated in previous steps

  
  #print(x)
  print('\nRequired solution is: ')
  for i in range(n):
    print('X%d = %0.2f' %(i,x[i]), end = '\t')  

  #x = list(x)


  return mtx,b, x


In [ ]:
#Test Case 0: 

B = np.array([[0, 2], [3,0]])
b = np.array([2,3])
U, y, x = gauss_elim(B, b)


Required solution is: 
X0 = 1.00	X1 = 1.00	

In [ ]:
#Test Case 1: 

B = np.array([[0, 2], [3,0]])
b = np.array([2,3])
U, y, x = gauss_elim(B, b)


Required solution is: 
X0 = 1.00	X1 = 1.00	

In [ ]:
#Test Case 2: 

A = np.array([[1, 2, 4], [2,1,3], [5,3,4]])
b = np.array([13,10,19])
U, y, x = gauss_elim(A, b)


Required solution is: 
X0 = 1.00	X1 = 2.00	X2 = 2.00	

In [ ]:
#Test Case 3: 

A = np.array([[0,1,1], [1,2,1], [2,7,9]])
b = np.array([13,10,19])
U, y, x = gauss_elim(A, b)

# **LU Decomposition**

In [ ]:
def lu_decomp(mtx): 
  ''' 
  Expected to have a square mtx as an input
  '''
  mtx = mtx.astype(np.float64)

  #### If not a square mtx, throw an error message
  if mtx.shape[0] != mtx.shape[1]:
    raise TypeError('Please provide a square matrix')
  
  n = mtx.shape[0] #Extract dimension
  L = np.zeros((n,n))  #Creating nxn zero-mtx

  #### Gauss can be implemented iff r(mtx) = r(mtx|b)
  if matrix_rank(mtx) != n: 
    raise TypeError('Please provide another square mtx, because yours is not a full rank square matrix.')

  for i in range(n): #Using pivot selection algorithm w/ regards to column
    abs_array = np.absolute(mtx[i:, i])   #Taking of abs values of each elemnts
    pivot_val = max(abs_array)   # select the  biggest value
    row_index = np.where(abs_array == pivot_val)[0][0] + i   #As we decrease the size of a matrix when selecting from the reduced column, to get the real row_index of the biggest element => need to append it with the reduced dimension
 
    mtx[[i,row_index]] = mtx[[row_index, i]]   #swap rows according to row_index determined by the biggest element in a column 

    if i == 0:   #At this step we don't tinker with L 
      P_orig = np.eye(n)    #creating nxn id_mtx, which would be our original permutation matrix 
      P_orig[[i,row_index]] = P_orig[[row_index, i]]   #changing rows according to th pivont element selection described abobe 
      P = copy.deepcopy(P_orig) 
      P_transient = P #make of the row-changed P_orig and save it to P_transient mtx which will be used below in i != 0 and i != n-1 iterations


    elif i != 0 and i != n-1:  # At this part we modify L in accordance with P_new which is transformed from row-exchaned P_orig
      P_orig = np.eye(n)    
      P_orig[[i,row_index]] = P_orig[[row_index, i]] 
      P_new = copy.deepcopy(P_orig)
      P = P_transient.dot(P_new)
      P_transient = P
      L = P_new.dot(L)

    
    ###Creating U from tinkering our original mtx and L 
    for j in range(i+1, n):
      L[i,i] = 1                 
      ratio = mtx[j,i]/mtx[i,i]                
      mtx[j] = mtx[j] - ratio*mtx[i]
      L[j,i] = ratio 

  L[n-1,n-1] = 1  # To fill the bottom-right corner with the appropriate 1 value for the L mtx


  print(P)
  print()
  print(L)
  print()
  print(mtx)

  return P,L,mtx




In [ ]:
B = np.array([[0, 2], [3,0]])
P,L,U = lu_decomp(B)

[[0. 1.]
 [1. 0.]]

[[1. 0.]
 [0. 1.]]

[[3. 0.]
 [0. 2.]]


In [ ]:
B = np.array([[1, 2], [3,1]])
p,l,u = lu(B)

print(p)
print('-----------')
print(l)
print('-----------')
print(u)

[[0. 1.]
 [1. 0.]]
-----------
[[1.         0.        ]
 [0.33333333 1.        ]]
-----------
[[3.         1.        ]
 [0.         1.66666667]]


In [ ]:
#Test Case


A = np.array([[0,1,1], [1,2,1], [2,7,9]])
#b = np.array([13,10,19])
P,L,U = lu_decomp(A)

print('-----------')
print('-----------')

print()
print(P.dot(A))
print('-----------')
print(L.dot(U))

[[0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]]

[[ 1.          0.          0.        ]
 [ 0.5         1.          0.        ]
 [ 0.         -0.66666667  1.        ]]

[[ 2.          7.          9.        ]
 [ 0.         -1.5        -3.5       ]
 [ 0.          0.         -1.33333333]]
-----------
-----------

[[2. 7. 9.]
 [1. 2. 1.]
 [0. 1. 1.]]
-----------
[[2. 7. 9.]
 [1. 2. 1.]
 [0. 1. 1.]]


In [ ]:
#Comparing with the LU-decomposition done by scipy package 

from scipy.linalg import lu
A = np.array([[0,1,1], [1,2,1], [2,7,9]])
p, l, u = lu(A)

print(p)
print()
print(L)
print()
print(u)

[[0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]]

[[ 1.          0.          0.        ]
 [ 0.5         1.          0.        ]
 [ 0.         -0.66666667  1.        ]]

[[ 2.          7.          9.        ]
 [ 0.         -1.5        -3.5       ]
 [ 0.          0.         -1.33333333]]


In [ ]:
A = np.array([[1,3,1,1,1], [9,1,1,1,2], [1,1,12,1,3], [1,1,3,1,4], [1,1,1,2,5]])
#b = np.array([1,6,4])
P,L,U = lu_decomp(A)

print('-----------')
print('-----------')

print()
print(P.dot(A))
print('-----------')
print(L.dot(U))

[[0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0.]]

[[1.         0.         0.         0.         0.        ]
 [0.11111111 1.         0.         0.         0.        ]
 [0.11111111 0.30769231 1.         0.         0.        ]
 [0.11111111 0.30769231 0.05298013 1.         0.        ]
 [0.11111111 0.30769231 0.22516556 0.30125523 1.        ]]

[[9.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  2.00000000e+00]
 [0.00000000e+00 2.88888889e+00 8.88888889e-01 8.88888889e-01
  7.77777778e-01]
 [0.00000000e+00 1.11022302e-16 1.16153846e+01 6.15384615e-01
  2.53846154e+00]
 [0.00000000e+00 1.05140326e-16 0.00000000e+00 1.58278146e+00
  4.40397351e+00]
 [0.00000000e+00 5.43498301e-17 0.00000000e+00 5.55111512e-17
  1.64016736e+00]]
-----------
-----------

[[ 9.  1.  1.  1.  2.]
 [ 1.  3.  1.  1.  1.]
 [ 1.  1. 12.  1.  3.]
 [ 1.  1.  1.  2.  5.]
 [ 1.  1.  3.  1.  4.]]
-----------
[[ 9.  1.  1.  1.  2.]
 [ 1.  3.  1.  1.  1.]
 [ 1.  1. 12.  1

In [ ]:
p,l,u = lu(A)

print(p)
print()
print(L)
print()
print(u)

[[0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0.]]

[[1.         0.         0.         0.         0.        ]
 [0.11111111 1.         0.         0.         0.        ]
 [0.11111111 0.30769231 1.         0.         0.        ]
 [0.11111111 0.30769231 0.05298013 1.         0.        ]
 [0.11111111 0.30769231 0.22516556 0.30125523 1.        ]]

[[ 9.          1.          1.          1.          2.        ]
 [ 0.          2.88888889  0.88888889  0.88888889  0.77777778]
 [ 0.          0.         11.61538462  0.61538462  2.53846154]
 [ 0.          0.          0.          1.58278146  4.40397351]
 [ 0.          0.          0.          0.          1.64016736]]
